In [1]:
import requests
import xml.etree.ElementTree as ET
from urllib3.exceptions import InsecureRequestWarning
import time
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

from Bio import Entrez
import ssl
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

import re

ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
gene_symbol = "CACNA1A"
email = ""

In [3]:
print("=== STEP 1: Searching ===")

# RefSeq 
search_term = f"{gene_symbol}[gene] AND Homo sapiens[organism] AND refseq[filter]"
print(f"query: {search_term}")

search_params = {
    'db': 'nucleotide',
    'term': search_term,
    'email': email,
    'retmax': 20,
    'retmode': 'xml'
}

search_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
response = requests.get(search_url, params=search_params, verify=False)


# ID 
root = ET.fromstring(response.text)
id_list = [id_elem.text for id_elem in root.findall('.//Id')]
count = root.find('.//Count')

print(f"#seq: {count.text if count is not None else 'Unknown'}")
print(f"#id: {len(id_list)}")
print("ID list:")
for i, seq_id in enumerate(id_list):
    print(f"  {i+1}. {seq_id}")


=== STEP 1: Searching ===
query: CACNA1A[gene] AND Homo sapiens[organism] AND refseq[filter]
#seq: 8
#id: 8
ID list:
  1. 2194972797
  2. 568815579
  3. 1851800768
  4. 1677538880
  5. 1677498315
  6. 1677498247
  7. 1677498183
  8. 224922780


In [4]:
print("\n=== STEP 2: esummary > summary ===")

summary_params = {
    'db': 'nucleotide',
    'id': ','.join(id_list),
    'email': email,
    'rettype': 'docsum',
    'retmode': 'xml'
}

summary_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi"
response = requests.get(summary_url, params=summary_params, verify=False)

root = ET.fromstring(response.text)


genomic_sequences = []  # NG_ - genomic region (intron )
transcript_sequences = []  # NM_ - mRNA transcript
chromosome_sequences = []  # NC_ - chromosome



for doc_sum in root.findall('.//DocSum'):
    seq_info = {'id': doc_sum.find('Id').text}
    
    for item in doc_sum.findall('Item'):
        name = item.get('Name', '')
        if name == 'AccessionVersion':
            seq_info['accession'] = item.text
        elif name == 'Title':
            seq_info['title'] = item.text
        elif name == 'Length':
            seq_info['length'] = int(item.text) if item.text.isdigit() else 0
    
    accession = seq_info.get('accession', '')
    
    if accession.startswith('NG_'):
        genomic_sequences.append(seq_info)
        print(f" GENOMIC: {seq_info['accession']} ({seq_info['length']:,} bp)")
        print(f"    → {seq_info['title']}")
        
    elif accession.startswith('NM_'):
        transcript_sequences.append(seq_info)
        print(f" TRANSCRIPT: {seq_info['accession']} ({seq_info['length']:,} bp)")
        print(f"    → {seq_info['title']}")
        
    elif accession.startswith('NC_'):
        chromosome_sequences.append(seq_info)
        print(f" CHROMOSOME: {seq_info['accession']} ({seq_info['length']:,} bp)")
        print(f"    → {seq_info['title']}")

print(f"- Genomic region (NG_): {len(genomic_sequences)}")
print(f"- Transcript (NM_): {len(transcript_sequences)}") 
print(f"- Chromosome (NC_): {len(chromosome_sequences)}")



=== STEP 2: esummary > summary ===
 CHROMOSOME: NC_060943.1 (61,707,364 bp)
    → Homo sapiens isolate CHM13 chromosome 19, alternate assembly T2T-CHM13v2.0
 CHROMOSOME: NC_000019.10 (58,617,616 bp)
    → Homo sapiens chromosome 19, GRCh38.p14 Primary Assembly
 TRANSCRIPT: NM_001127221.2 (8,645 bp)
    → Homo sapiens calcium voltage-gated channel subunit alpha1 A (CACNA1A), transcript variant 3, mRNA
 TRANSCRIPT: NM_023035.3 (8,665 bp)
    → Homo sapiens calcium voltage-gated channel subunit alpha1 A (CACNA1A), transcript variant 2, mRNA
 TRANSCRIPT: NM_001174080.2 (8,651 bp)
    → Homo sapiens calcium voltage-gated channel subunit alpha1 A (CACNA1A), transcript variant 5, mRNA
 TRANSCRIPT: NM_001127222.2 (8,647 bp)
    → Homo sapiens calcium voltage-gated channel subunit alpha1 A (CACNA1A), transcript variant 4, mRNA
 TRANSCRIPT: NM_000068.4 (8,660 bp)
    → Homo sapiens calcium voltage-gated channel subunit alpha1 A (CACNA1A), transcript variant 1, mRNA
 GENOMIC: NG_011569.1 (307,01

In [5]:
print("\n=== STEP 3: efetch > genomic sequence details ===")


genomic_accession = "NG_011569.1"
handle = Entrez.efetch(db="nucleotide", id=genomic_accession, rettype="gb", retmode="text")
record = SeqIO.read(handle, "genbank")
handle.close()

print(f"  Accession: {record.id}")
print(f"  Length: {len(record.seq)} bp")
print(f"  Definition: {record.description}")



=== STEP 3: efetch > genomic sequence details ===


/BiO/Live/jolee/.local/lib/python3.8/site-packages/Bio/Entrez/__init__.py:723: UserWarning: 
            Email address is not specified.

            To make use of NCBI's E-utilities, NCBI requires you to specify your
            email address with each request.  As an example, if your email address
            is A.N.Other@example.com, you can specify it as follows:
               from Bio import Entrez
               Entrez.email = 'A.N.Other@example.com'
            In case of excessive usage of the E-utilities, NCBI will attempt to contact
            a user at the email address provided before blocking access to the
            E-utilities.
  warnings.warn(


  Accession: NG_011569.1
  Length: 307019 bp
  Definition: Homo sapiens calcium voltage-gated channel subunit alpha1 A (CACNA1A), RefSeqGene (LRG_7) on chromosome 19


In [6]:
ftype = []
for feature in record.features:
    #print(feature.type)
    ftype.append(feature.type)
    
print(list(set(ftype)))


['mRNA', 'CDS', 'misc_feature', 'exon', 'gene', 'source']


In [7]:
for feature in record.features:
    if feature.type == "gene":
        print(feature)

type: gene
location: [5000:305019](+)
qualifiers:
    Key: db_xref, Value: ['GeneID:773', 'HGNC:HGNC:1388', 'MIM:601011']
    Key: gene, Value: ['CACNA1A']
    Key: gene_synonym, Value: ['APCA; BI; CACNL1A4; CAV2.1; DEE42; EA2; EIEE42; FHM; HPCA; MHP; MHP1; SCA6']
    Key: note, Value: ['calcium voltage-gated channel subunit alpha1 A']



In [8]:
## NM_001127221.1 transcript id  check
for feature in record.features:
    if feature.type == "mRNA":
        print(feature)

type: mRNA
location: join{[5000:5529](+), [56248:56354](+), [58445:58585](+), [139681:139773](+), [145991:146144](+), [151661:151855](+), [175551:175655](+), [176967:177083](+), [178535:178592](+), [181124:181217](+), [194139:194349](+), [198679:198792](+), [202932:203045](+), [203209:203341](+), [203606:203679](+), [207576:207694](+), [207847:207915](+), [210804:210911](+), [212107:212917](+), [224494:224958](+), [226254:226393](+), [228064:228194](+), [234332:234392](+), [235504:235611](+), [248627:248727](+), [249850:250011](+), [251759:251897](+), [253909:254111](+), [256201:256366](+), [258359:258470](+), [266195:266279](+), [275730:275847](+), [276186:276252](+), [276424:276540](+), [279600:279751](+), [281251:281379](+), [282665:282762](+), [286688:286794](+), [296852:296960](+), [297127:297228](+), [298720:298830](+), [298938:299077](+), [299244:299358](+), [300808:300844](+), [301962:302149](+), [302451:302705](+), [303412:305019](+)}
qualifiers:
    Key: db_xref, Value: ['Gen

In [9]:
## Domain 
import pandas as pd
from Bio import SeqIO
import re


cds_exons = None
for feature in record.features:
    if feature.type == "CDS":
        if hasattr(feature.location, 'parts'):
            cds_exons = [(int(part.start), int(part.end)) for part in feature.location.parts]
        else:
            cds_exons = [(int(feature.location.start), int(feature.location.end))]
        break


# Genomic → Coding position
def genomic_to_coding(genomic_pos, exon_list):
    if exon_list is None:
        return None
    cumulative_length = 0
    for exon_start, exon_end in exon_list:
        if genomic_pos >= exon_start and genomic_pos < exon_end:
            offset = genomic_pos - exon_start
            return cumulative_length + offset + 1
        cumulative_length += (exon_end - exon_start)
    return None

# Genomic → AA
def genomic_to_aa(genomic_pos, exon_list):
    coding_pos = genomic_to_coding(genomic_pos, exon_list)
    if coding_pos:
        return (coding_pos - 1) // 3 + 1
    return None

# Genomic → Exon 
def find_exon_number(genomic_pos, exon_list):
    if exon_list is None:
        return None
    for i, (exon_start, exon_end) in enumerate(exon_list, 1):
        if genomic_pos >= exon_start and genomic_pos < exon_end:
            return i
    return None

# Feature 
def classify_feature(note):

    note_lower = note.lower()
    
    # Region: 
    if 'Region:' in note or 'region:' in note_lower:
        # Domain I, II, III, IV
        match = re.search(r'Region:\s*([IVX]+)\s*$', note)
        if match:
            return f'Domain {match.group(1)}'
        
        # Disordered region
        if 'disordered' in note_lower:
            return 'Disordered region'
        
        # Binding region
        if 'binding' in note_lower:
            if 'beta subunit' in note_lower:
                return 'Beta subunit binding'
            else:
                return 'Binding region'
        
        # other Region
        match = re.search(r'Region:\s*(.+?)\.', note)
        if match:
            region_desc = match.group(1).strip()
            return f'Region: {region_desc}'
    
    # Transmembrane
    if 'transmembrane' in note_lower:
        return 'Transmembrane'
    
    # Phosphorylation
    if 'phospho' in note_lower:
        if 'phosphoserine' in note_lower:
            return 'Phosphorylation (Ser)'
        elif 'phosphothreonine' in note_lower:
            return 'Phosphorylation (Thr)'
        elif 'phosphotyrosine' in note_lower:
            return 'Phosphorylation (Tyr)'
        else:
            return 'Phosphorylation'
    
    # Glycosylation
    if 'glycosylation' in note_lower or 'n-linked' in note_lower:
        return 'Glycosylation'
    
    # Modified residue
    if 'modified residue' in note_lower:
        return 'Modified residue'
    
    # Binding site
    if 'binds to' in note_lower or 'binding' in note_lower:
        if 'omega-aga' in note_lower or 'agatoxin' in note_lower:
            return 'Toxin binding'
        else:
            return 'Binding site'
    
    # Active site
    if 'active site' in note_lower:
        return 'Active site'
    
    # Metal binding
    if 'metal' in note_lower:
        return 'Metal binding'
    
    # Disulfide bond
    if 'disulfide' in note_lower:
        return 'Disulfide bond'
    
    # 기타
    return 'Other'

# Feature 
def extract_description(note):
    # "propagated from..." 
    desc = re.sub(r'propagated from.*?;\s*', '', note)
    # "/evidence=..." 
    desc = re.sub(r'/evidence=.*$', '', desc)
    return desc.strip()


data = []
feature_count = 0

for feature in record.features:
    if feature.type == "misc_feature":
        feature_count += 1
        location = feature.location
        note = feature.qualifiers.get('note', [''])[0]
        
        if hasattr(location, 'parts'):
            coords = [(int(part.start), int(part.end)) for part in location.parts]
            start = min(s for s, e in coords)
            end = max(e for s, e in coords)
            num_parts = len(coords)
        else:
            coords = [(int(location.start), int(location.end))]
            start, end = coords[0]
            num_parts = 1
        
        # Coding position 
        coding_start = genomic_to_coding(start, cds_exons) if cds_exons else None
        coding_end = genomic_to_coding(end - 1, cds_exons) if cds_exons else None
        
        # c. notation
        if coding_start and coding_end:
            if coding_start == coding_end:
                c_notation = f"c.{coding_start}"
            else:
                c_notation = f"c.{coding_start}_{coding_end}"
        else:
            c_notation = ""
        
        # AA
        aa_start = genomic_to_aa(start, cds_exons) if cds_exons else None
        aa_end = genomic_to_aa(end - 1, cds_exons) if cds_exons else None
        aa_length = (aa_end - aa_start + 1) if (aa_start and aa_end) else None
        
        # p. notation
        if aa_start and aa_end:
            if aa_start == aa_end:
                p_notation = f"p.{aa_start}"
            else:
                p_notation = f"p.{aa_start}_{aa_end}"
        else:
            p_notation = ""
        
        # Exon 
        exon_start = find_exon_number(start, cds_exons) if cds_exons else None
        exon_end = find_exon_number(end - 1, cds_exons) if cds_exons else None
        
        if exon_start and exon_end:
            if exon_start == exon_end:
                exon_location = f"Exon {exon_start}"
            else:
                exon_location = f"Exon {exon_start}-{exon_end}"
        else:
            exon_location = ""
        
        # Feature 
        feature_type = classify_feature(note)
        
        description = extract_description(note)
        
        data.append({
            'ID': feature_count,
            'Feature_Type': feature_type,
            'Exon_Location': exon_location,
            'c_notation': c_notation,
            'p_notation': p_notation,
            'AA_Length': aa_length,
            'Description': description,
            'Full_Note': note
        })

# DataFrame 
df = pd.DataFrame(data)
df.head()

,ID,Feature_Type,Exon_Location,c_notation,p_notation,AA_Length,Description,Full_Note
0,1,Domain I,Exon 1-8,c.253_1089,p.85_363,279,Region: I,propagated from UniProtKB/Swiss-Prot (O00555.3...
1,2,Transmembrane,Exon 2,c.295_351,p.99_117,19,transmembrane region,propagated from UniProtKB/Swiss-Prot (O00555.3...
2,3,Transmembrane,Exon 3,c.406_465,p.136_155,20,transmembrane region,propagated from UniProtKB/Swiss-Prot (O00555.3...
3,4,Transmembrane,Exon 3-4,c.502_555,p.168_185,18,transmembrane region,propagated from UniProtKB/Swiss-Prot (O00555.3...
4,5,Transmembrane,Exon 4,c.571_627,p.191_209,19,transmembrane region,propagated from UniProtKB/Swiss-Prot (O00555.3...


In [10]:
### CDS
cds_sequence = ""
protein_sequence = ""
codon_number=""
protein_id=""
            
for feature in record.features:
    if feature.type == "CDS":
        # CDS 서열 추출
        if hasattr(feature.location, 'parts'):
            for part in feature.location.parts:
                cds_sequence += str(record.seq[part.start:part.end])
        else:
            cds_sequence += str(record.seq[feature.location.start:feature.location.end])

        ## feature.qualifiers.key : codon start, protine id, translation 
        codon_number = feature.qualifiers.get('codon_start',[''])[0]
        protein_id = feature.qualifiers.get('protein_id',[''])[0]
        protein_sequence = feature.qualifiers.get('translation', [''])[0]


In [11]:
### intron
exons = []
introns = []

# Exon feature 
for feature in record.features:
    if (feature.type == "exon" and 
        gene_symbol.lower() in str(feature.qualifiers.get("gene", "")).lower()):
        exons.append({
            'number': len(exons) + 1,
            'start': int(feature.location.start),
            'end': int(feature.location.end),
            'sequence': str(record.seq[feature.location.start:feature.location.end])
        })

# Intron 
if len(exons) > 1:
    exons.sort(key=lambda x: x['start'])
    for i in range(len(exons) - 1):
        intron_start = exons[i]['end']
        intron_end = exons[i + 1]['start']
        if intron_end > intron_start:
            intron_seq = str(record.seq[intron_start:intron_end])
            introns.append({
                'number': i + 1,
                'start': intron_start,
                'end': intron_end,
                'sequence': intron_seq,
                'donor_site': intron_seq[:2] if len(intron_seq) >= 2 else "",
                'acceptor_site': intron_seq[-2:] if len(intron_seq) >= 2 else ""
            })


## Variants

In [12]:
snp_cases = [
    {
        'gene_symbol': 'CACNA1A',
        'transcript_id': 'NM_001127221.1',
        'cmutation': 'c.C757T', 
        'pmutation': 'p.H253Y'
    }]

indel_cases = [
    {
        'gene_symbol': 'CACNA1A',
        'transcript_id': 'NM_001127221.1',
        'cmutation': 'c.3415delinsCA',
        'pmutation': 'p.K1139Qfs*6'
        
    }
]

intron_retain_cases = [
        {
        'gene_symbol': 'CACNA1A',
        'transcript_id': 'NM_001127221.1',
        'cmutation': 'c.4953+1G>A',
        'pmutation': 'splicing'    }
]

In [13]:
def parse_mutation(mutation: str):
    """
    parsing (HGVS format)
    
    Args:
        mutation: ex(: c.123A>G, c.123_125del, c.4950+1G>A)
        
    Returns:
        dictionary
    """
   
    if mutation.startswith('c.'):
        mutation = mutation[2:].strip('()')
        
        # splice site : 4950+1G>A, 123-2T>C
        splice_pattern = r'(\d+)([\+\-])(\d+)([ATCG])>([ATCG])'
        match = re.match(splice_pattern, mutation)
        if match:
            exon_pos = int(match.group(1))
            direction = match.group(2)
            intron_offset = int(match.group(3))
            ref = match.group(4)
            alt = match.group(5)

            return {
                'format': 'c.',
                'type': 'splice_site',
                'exon_position': exon_pos,
                'direction': direction,
                'intron_offset': intron_offset,
                'ref': ref,
                'alt': alt,
                'splice_type': 'donor' if direction == '+' and intron_offset <= 2 else 
                              'acceptor' if direction == '-' and intron_offset <= 2 else 
                              'deep_intronic'
            }

        # substitution: 123A>G 
        sub_pattern = r'^(?:(\d+)([ATCG])>([ATCG]))|(?:([ATCG])(\d+)([ATCG]))$'
        match = re.match(sub_pattern, mutation, re.IGNORECASE)
        if match:
            if match.group(1):  # 123A>G 형태
                return {
                    'format': 'c.',
                    'type': 'substitution',
                    'position': int(match.group(1)) - 1,  # 0-based
                    'ref': match.group(2).upper(),
                    'alt': match.group(3).upper()
                }
            else:  # C757T 
                return {
                    'format': 'c.',
                    'type': 'substitution',
                    'position': int(match.group(5)) - 1,  # 0-based
                    'ref': match.group(4).upper(),
                    'alt': match.group(6).upper()
                }

        # delins: 123delinsGA, 123_125delinsATG
        delins_pattern = r'^(\d+)(?:_(\d+))?delins([ATCG]+)$'
        match = re.match(delins_pattern, mutation)
        if match:
            start = int(match.group(1)) - 1  # 0-based
            end = int(match.group(2)) - 1 if match.group(2) else start
            alt = match.group(3)
            return {
                'format': 'c.',
                'type': 'delins',
                'start': start,
                'end': end,
                'alt': alt
            }
        
        # deletion: 123del, 123_125del
        del_pattern = r'^(\d+)(?:_(\d+))?del([ATCG]*)$'
        match = re.match(del_pattern, mutation)
        if match:
            start = int(match.group(1)) - 1  # 0-based
            end = int(match.group(2)) - 1 if match.group(2) else start
            return {
                'format': 'c.',
                'type': 'deletion',
                'start': start,
                'end': end
            }

        # insertion: 123_124insATG
        ins_pattern = r'^(\d+)_(\d+)ins([ATCG]+)$'
        match = re.match(ins_pattern, mutation)
        if match:
            return {
                'format': 'c.',
                'type': 'insertion',
                'position': int(match.group(1)),
                'alt': match.group(3)
            }

    elif mutation.startswith('p.'):
        p_mutation = mutation[2:].strip('()')

        # 아미노산 코드 매핑
        AA_CODES = {
            'Ala': 'A', 'Arg': 'R', 'Asn': 'N', 'Asp': 'D', 'Cys': 'C',
            'Gln': 'Q', 'Glu': 'E', 'Gly': 'G', 'His': 'H', 'Ile': 'I',
            'Leu': 'L', 'Lys': 'K', 'Met': 'M', 'Phe': 'F', 'Pro': 'P',
            'Ser': 'S', 'Thr': 'T', 'Trp': 'W', 'Tyr': 'Y', 'Val': 'V',
            'Ter': '*'
        }

        def to_one_letter(aa):
            if len(aa) == 3:
                return AA_CODES.get(aa, aa)
            return aa
           
        fs_pattern = r'^([A-Z*])(\d+)([A-Z*])fs\*(\d+)$'
        match = re.match(fs_pattern, p_mutation)
        if match:
            return {
                'format': 'p.',
                'type': 'frameshift',
                'position': int(match.group(2)) - 1,  
                'ref': match.group(1),
                'alt': match.group(3),
                'stop_pos_offset': int(match.group(4))
            }
        
        # Deletion: E1291del
        del_pattern = r'^([A-Z*])(\d+)del$'
        match = re.match(del_pattern, p_mutation)    
        if match:
            return {
                'format': 'p.',
                'type': 'deletion',
                'position': int(match.group(2)) - 1,  # 1-based
                'ref': match.group(1),
                'alt': 'del'
            }
        
        # Nonsense : R1857X, R1819X
        nonsense_pattern = r'^([A-Z][a-z]{2}|[A-Z])(\d+)(\*|X|Ter)$'
        match = re.match(nonsense_pattern, p_mutation)
        if match:
            return {
                'format': 'p.',
                'type': 'nonsense',
                'position': int(match.group(2)) - 1,  # 1-based
                'ref': to_one_letter(match.group(1))
            }
        
        # Substitution: 
        sub_pattern = r'^([A-Z][a-z]{2}|[A-Z*])(\d+)([A-Z][a-z]{2}|[A-Z*])$'
        match = re.match(sub_pattern, p_mutation)
        if match:
            return {
                'format': 'p.',
                'type': 'substitution',
                'position': int(match.group(2)) -1 ,  # 1-based
                'ref': to_one_letter(match.group(1)),
                'alt': to_one_letter(match.group(3))
            }

        # 미확인/복잡 변이
        return {'format': 'p.', 'type': 'unknown', 'raw': p_mutation}

    else:
        return {'format': 'unknown', 'type': 'raw', 'raw': mutation}


In [14]:
pmutations_list = [item['pmutation'] for item in snp_cases]
seq_alt_list = []

for pmut in pmutations_list:
    seq_alt = protein_sequence
    ppar = parse_mutation(pmut)
    

    pos_0based = ppar['position'] 

    print(f"variants: {pmut}, position(0-based): {ppar['position']}, alt: {ppar['alt']}")
    
    seq_alt_new = seq_alt[:pos_0based] + ppar['alt'] + seq_alt[pos_0based + 1:]

    print(f"before (0-based {pos_0based}): {seq_alt[pos_0based]}")
    print(f"after (0-based {pos_0based}): {seq_alt_new[pos_0based]}")
    print("--------------------")
    print(seq_alt_new)
    

    seq_alt_list.append(seq_alt_new)

variants: p.H253Y, position(0-based): 252, alt: Y
before (0-based 252): H
after (0-based 252): Y
--------------------
MARFGDEMPARYGGGGSGAAAGVVVGSGGGRGAGGSRQGGQPGAQRMYKQSMAQRARTMALYNPIPVRQNCLTVNRSLFLFSEDNVVRKYAKKITEWPPFEYMILATIIANCIVLALEQHLPDDDKTPMSERLDDTEPYFIGIFCFEAGIKIIALGFAFHKGSYLRNGWNVMDFVVVLTGILATVGTEFDLRTLRAVRVLRPLKLVSGIPSLQVVLKSIMKAMIPLLQIGLLLFFAILIFAIIGLEFYMGKFYTTCFEEGTDDIQGESPAPCGTEEPARTCPNGTKCQPYWEGPNNGITQFDNILFAVLTVFQCITMEGWTDLLYNSNDASGNTWNWLYFIPLIIIGSFFMLNLVLGVLSGEFAKERERVENRRAFLKLRRQQQIERELNGYMEWISKAEEVILAEDETDGEQRHPFDGALRRTTIKKSKTDLLNPEEAEDQLADIASVGSPFARASIKSAKLENSTFFHKKERRMRFYIRRMVKTQAFYWTVLSLVALNTLCVAIVHYNQPEWLSDFLYYAEFIFLGLFMSEMFIKMYGLGTRPYFHSSFNCFDCGVIIGSIFEVIWAVIKPGTSFGISVLRALRLLRIFKVTKYWASLRNLVVSLLNSMKSIISLLFLLFLFIVVFALLGMQLFGGQFNFDEGTPPTNFDTFPAAIMTVFQILTGEDWNEVMYDGIKSQGGVQGGMVFSIYFIVLTLFGNYTLLNVFLAIAVDNLANAQELTKDEQEEEEAANQKLALQKAKEVAEVSPLSAANMSIAVKEQQKNQKPAKSVWEQRTSEMRKQNLLASREALYNEMDPDERWKAAYTRHLRPDMKTHLDRPLVVDPQENRNNNTNKSRAAEPTVDQRLGQQRAEDFLRKQARYHDRARDPSGSAGLDARR

In [15]:
for case in indel_cases: 
    cmut = case['cmutation']
    pmut = case['pmutation']
     
    seq_alt_cds = cds_sequence
    seq_alt_pro = protein_sequence
    
    cpar = parse_mutation(cmut)
    ppar = parse_mutation(pmut)

    print("cDNA mutation:" + cmut)
    print("protein muation:" +pmut)

    
    if ppar['type'] == "frameshift":
        
        if cpar['type'] == 'deletion':
            cpos_0based = cpar['start']

            if cpar['start'] == cpar['end']:
                seq_alt_cds_new = seq_alt_cds[:cpos_0based]+seq_alt_cds[cpos_0based+1:]

            if cpar['start'] != cpar['end']:
                cpos_ebased = cpar['end']
                seq_alt_cds_new = seq_alt_cds[:cpos_0based]+seq_alt_cds[cpos_ebased+1:]
                
        if cpar['type'] == 'delins':
            cpos_0based = cpar['start']
            seq_alt_cds_new = seq_alt_cds[:cpos_0based]+cpar['alt']+seq_alt_cds[cpos_0based+1:]


    
        cds_new = SeqRecord(seq_alt_cds_new)
        protein_new = cds_new.translate(to_stop=True)
        seq_alt_protein_new = str(protein_new.seq)
        lento=len(seq_alt_protein_new[ppar['position']:])
        
        if lento +1 == ppar['stop_pos_offset']:
            print("Truncated sequence!")
#            print(seq_alt_protein_new[ppar['position']:])
#            len(seq_alt_protein_new[ppar['position']:])
            print("--------------------")
            print(seq_alt_protein_new)


cDNA mutation:c.3415delinsCA
protein muation:p.K1139Qfs*6
Truncated sequence!
--------------------
MARFGDEMPARYGGGGSGAAAGVVVGSGGGRGAGGSRQGGQPGAQRMYKQSMAQRARTMALYNPIPVRQNCLTVNRSLFLFSEDNVVRKYAKKITEWPPFEYMILATIIANCIVLALEQHLPDDDKTPMSERLDDTEPYFIGIFCFEAGIKIIALGFAFHKGSYLRNGWNVMDFVVVLTGILATVGTEFDLRTLRAVRVLRPLKLVSGIPSLQVVLKSIMKAMIPLLQIGLLLFFAILIFAIIGLEFYMGKFHTTCFEEGTDDIQGESPAPCGTEEPARTCPNGTKCQPYWEGPNNGITQFDNILFAVLTVFQCITMEGWTDLLYNSNDASGNTWNWLYFIPLIIIGSFFMLNLVLGVLSGEFAKERERVENRRAFLKLRRQQQIERELNGYMEWISKAEEVILAEDETDGEQRHPFDGALRRTTIKKSKTDLLNPEEAEDQLADIASVGSPFARASIKSAKLENSTFFHKKERRMRFYIRRMVKTQAFYWTVLSLVALNTLCVAIVHYNQPEWLSDFLYYAEFIFLGLFMSEMFIKMYGLGTRPYFHSSFNCFDCGVIIGSIFEVIWAVIKPGTSFGISVLRALRLLRIFKVTKYWASLRNLVVSLLNSMKSIISLLFLLFLFIVVFALLGMQLFGGQFNFDEGTPPTNFDTFPAAIMTVFQILTGEDWNEVMYDGIKSQGGVQGGMVFSIYFIVLTLFGNYTLLNVFLAIAVDNLANAQELTKDEQEEEEAANQKLALQKAKEVAEVSPLSAANMSIAVKEQQKNQKPAKSVWEQRTSEMRKQNLLASREALYNEMDPDERWKAAYTRHLRPDMKTHLDRPLVVDPQENRNNNTNKSRAAEPTVDQRLGQQRAEDFLRKQARYHDRARDPSGSAGLDARRPWAGSQEAELSREGPYGRE

/BiO/Live/jolee/.local/lib/python3.8/site-packages/Bio/SeqRecord.py:229: BiopythonDeprecationWarning: Using a string as the sequence is deprecated and will raise a TypeError in future. It has been converted to a Seq object.
  warnings.warn(
/BiO/Live/jolee/.local/lib/python3.8/site-packages/Bio/Seq.py:2880: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


In [16]:
## splicing 

In [17]:
def find_exon_from_cds_position(cds_position, record, gene_symbol):

    for feature in record.features:
        if feature.type == "CDS":
            cumulative_length = 0
            exon_number = 0
            
            if hasattr(feature.location, 'parts'):
                parts = feature.location.parts
            else:
                parts = [feature.location]
            
            for part in parts:
                exon_number += 1
                part_length = len(part)
                
                if cumulative_length < cds_position <= cumulative_length + part_length:
                    position_in_exon = cds_position - cumulative_length
                    return {
                        'exon_number': exon_number,
                        'position_in_exon': position_in_exon,
                        'genomic_start': int(part.start),
                        'genomic_end': int(part.end),
                        'genomic_position': int(part.start) + position_in_exon - 1 if feature.strand == 1 
                                           else int(part.end) - position_in_exon + 1
                    }
                
                cumulative_length += part_length
            
            if cds_position == cumulative_length:
                return {
                    'exon_number': exon_number,
                    'position_in_exon': part_length,
                    'genomic_start': int(parts[-1].start),
                    'genomic_end': int(parts[-1].end)
                }
    
    return None

In [18]:

for case in intron_retain_cases: 
    cmut = case['cmutation']
    pmut = case['pmutation']
     
    seq_alt_cds = cds_sequence
    seq_alt_pro = protein_sequence
    
    cpar = parse_mutation(cmut)
    ppar = parse_mutation(pmut)

    print("cDNA mutation:  ===========")
    print(cmut)
    print( cpar)
    print("protein muation:  ===========")
    print(pmut)
    print(ppar)
    
    cpos_0based = cpar['exon_position']
    num_intron = find_exon_from_cds_position(cpos_0based, record, case['gene_symbol'])['exon_number']-1
    seq_alt_cds_new = seq_alt_cds[:cpos_0based]+cpar['alt']+introns[num_intron]['sequence'][1:]
    
    cds_record= SeqRecord(Seq(seq_alt_cds[:cpos_0based]))
    protein_translated = str(cds_record.seq.translate())
    print(protein_translated)

    print("--------variants---------")
    mut_cds_record = SeqRecord(Seq(seq_alt_cds_new))
    mut_protein_translated = str(mut_cds_record.seq.translate(to_stop=True))
    print(mut_protein_translated)


cDNA mutation:  ===========
c.4953+1G>A
{'format': 'c.', 'type': 'splice_site', 'exon_position': 4953, 'direction': '+', 'intron_offset': 1, 'ref': 'G', 'alt': 'A', 'splice_type': 'donor'}
protein muation:  ===========
splicing
{'format': 'unknown', 'type': 'raw', 'raw': 'splicing'}
MARFGDEMPARYGGGGSGAAAGVVVGSGGGRGAGGSRQGGQPGAQRMYKQSMAQRARTMALYNPIPVRQNCLTVNRSLFLFSEDNVVRKYAKKITEWPPFEYMILATIIANCIVLALEQHLPDDDKTPMSERLDDTEPYFIGIFCFEAGIKIIALGFAFHKGSYLRNGWNVMDFVVVLTGILATVGTEFDLRTLRAVRVLRPLKLVSGIPSLQVVLKSIMKAMIPLLQIGLLLFFAILIFAIIGLEFYMGKFHTTCFEEGTDDIQGESPAPCGTEEPARTCPNGTKCQPYWEGPNNGITQFDNILFAVLTVFQCITMEGWTDLLYNSNDASGNTWNWLYFIPLIIIGSFFMLNLVLGVLSGEFAKERERVENRRAFLKLRRQQQIERELNGYMEWISKAEEVILAEDETDGEQRHPFDGALRRTTIKKSKTDLLNPEEAEDQLADIASVGSPFARASIKSAKLENSTFFHKKERRMRFYIRRMVKTQAFYWTVLSLVALNTLCVAIVHYNQPEWLSDFLYYAEFIFLGLFMSEMFIKMYGLGTRPYFHSSFNCFDCGVIIGSIFEVIWAVIKPGTSFGISVLRALRLLRIFKVTKYWASLRNLVVSLLNSMKSIISLLFLLFLFIVVFALLGMQLFGGQFNFDEGTPPTNFDTFPAAIMTVFQILTGEDWNEVMYDGIKSQGGVQGGMVFSIYFIVLTLFGNYTLLNVFLAIAVDN

/BiO/Live/jolee/.local/lib/python3.8/site-packages/Bio/SeqFeature.py:230: BiopythonDeprecationWarning: Please use .location.strand rather than .strand
  warnings.warn(
